In [71]:
import pandas as pd
import numpy as np
import datetime
import quandl
import matplotlib.pyplot as plt

In [72]:
#import the quandl key
quandl.ApiConfig.api_key = "HYs_ygeP49iq5pzoQVh6"

In [74]:
# Define 10 stocks for which we want to do create a portfolio
df={'stocks':['NSE/AXISBANK','NSE/BHARTIARTL','NSE/EICHERMOT','NSE/HDFCBANK','NSE/INDUSINDBK','NSE/MARUTI','NSE/RELIANCE','NSE/TCS','NSE/VEDL','NSE/YESBANK']}
all_stocks = pd.DataFrame(data=df)


In [32]:
#Get all stocks data as present in our list above from quandl
#The below mentioned function will create a multi level index. Essentially the data would be contained in the below format in the dataframe
#                Open    Close    High   Low   Adj Close   Volume   Turnover
# <Stock> <Date1> v1     v1       v1     v1    v1           v1       v1
#         <Date2> v2     v2       v2     v2    v2           v2       v2
#         ...     ...    ...      ...    ...   ...          ...      ....

# The first index is <Stock> and the second index is <Date>

def getAllStocksData(allStocks,startDate,endDate):
    def getIndividualStockData(stock):
        return quandl.get(stock,api_key='HYs_ygeP49iq5pzoQVh6',start_date=startDate,end_date=endDate)
    datas=map(getIndividualStockData,allStocks)
    return (pd.concat(datas,keys=allStocks,names=['Stock','Date']))

In [75]:
all_stocks.iloc[0]

stocks    NSE/AXISBANK
Name: 0, dtype: object

In [73]:
#Function for computing commodity channel index
def CCI(allStocks,data, ndays):
    def individualStockCCI(stock):
        TP = (data.loc[stock]['High'] + data.loc[stock]['Low'] + data.loc[stock]['Close']) / 3 
        data.loc[stock]['CCI'] = pd.Series((TP - TP.rolling(window=ndays).mean()) / (0.015 * TP.rolling(window=ndays).mean()),name = 'CCI') 
        return data
    #map function internally does apply for all elements belonging to a given list with the function being called.
    datas=map(individualStockCCI,allStocks)
    return (pd.concat(datas,allStocks,names=['Stock','Date']))

In [76]:
#data = quandl.get(df['stocks'],api_key='HYs_ygeP49iq5pzoQVh6',start_date="2013-01-01", end_date="2018-01-01")

In [77]:
#Get data for all stocks in the portfolio for the last 5 years
allStockData = getAllStocksData(df['stocks'],"2013-01-01","2018-01-01")

In [78]:
allStockData.index

MultiIndex(levels=[['NSE/AXISBANK', 'NSE/BHARTIARTL', 'NSE/EICHERMOT', 'NSE/HDFCBANK', 'NSE/INDUSINDBK', 'NSE/MARUTI', 'NSE/RELIANCE', 'NSE/TCS', 'NSE/VEDL', 'NSE/YESBANK'], [2013-01-01 00:00:00, 2013-01-02 00:00:00, 2013-01-03 00:00:00, 2013-01-04 00:00:00, 2013-01-07 00:00:00, 2013-01-08 00:00:00, 2013-01-09 00:00:00, 2013-01-10 00:00:00, 2013-01-11 00:00:00, 2013-01-14 00:00:00, 2013-01-15 00:00:00, 2013-01-16 00:00:00, 2013-01-17 00:00:00, 2013-01-18 00:00:00, 2013-01-21 00:00:00, 2013-01-22 00:00:00, 2013-01-23 00:00:00, 2013-01-24 00:00:00, 2013-01-25 00:00:00, 2013-01-28 00:00:00, 2013-01-29 00:00:00, 2013-01-30 00:00:00, 2013-01-31 00:00:00, 2013-02-01 00:00:00, 2013-02-04 00:00:00, 2013-02-05 00:00:00, 2013-02-06 00:00:00, 2013-02-07 00:00:00, 2013-02-08 00:00:00, 2013-02-11 00:00:00, 2013-02-12 00:00:00, 2013-02-13 00:00:00, 2013-02-14 00:00:00, 2013-02-15 00:00:00, 2013-02-18 00:00:00, 2013-02-19 00:00:00, 2013-02-20 00:00:00, 2013-02-21 00:00:00, 2013-02-22 00:00:00, 2013-0

In [80]:
#practice to rename columns if required
def chomp(col,s):
    return col[col.index('/')+1:col.index('-')]+col[col.index(s)+2:]

In [81]:
#print(chomp('NSE/INDUSINDBK - Last','-'))

INDUSINDBK Last


In [83]:

#data.rename(columns=lambda x: chomp(x,'-'),inplace=True)

In [32]:
#data.to_csv("C:/BigData/stockData.csv", sep='\t')

In [79]:
#Compute the CCI across the entire dataset
# Compute the Commodity Channel Index(CCI) for NIFTY based on the 20-day Moving average
n = 20
#Create an empty column to store the tech indicator related to CCI
allStockData = allStockData.reindex( columns = allStockData.columns.tolist() + ['CCI'])
NIFTY_CCI = CCI(df['stocks'],allStockData, n)
CCI = NIFTY_CCI['CCI']

In [85]:
#TP = (allStockData.loc['NSE/BHARTIARTL']['High'] + allStockData.loc['NSE/BHARTIARTL']['Low'] + allStockData.loc['NSE/BHARTIARTL']['Close'])/3
#allStockData.loc['NSE/BHARTIARTL']['CCI'] = pd.Series((TP - TP.rolling(window=20).mean()) / (0.015 * TP.rolling(window=20).mean()),name = 'CCI') 
allStockData.loc['NSE/AXISBANK']

,Open,High,Low,Last,Close,Total Trade Quantity,Turnover (Lacs),CCI
Date,,,,,,,,
2013-01-01,1369.70,1371.50,1360.50,1362.00,1362.70,668722.0,9130.71,NaN
2013-01-02,1371.15,1394.40,1366.65,1386.75,1386.85,1433645.0,19859.63,NaN
2013-01-03,1394.00,1396.50,1383.25,1390.00,1389.95,950039.0,13207.97,NaN
2013-01-04,1382.45,1386.95,1370.00,1377.25,1378.95,1252580.0,17263.78,NaN
2013-01-07,1385.00,1387.70,1365.50,1366.00,1372.85,1143170.0,15717.40,NaN
2013-01-08,1368.80,1369.00,1343.05,1362.70,1360.30,1218122.0,16521.91,NaN
2013-01-09,1352.25,1361.95,1344.10,1351.00,1349.65,1577025.0,21291.63,NaN
2013-01-10,1359.00,1376.80,1345.50,1369.15,1368.00,1371873.0,18630.88,NaN
2013-01-11,1372.10,1374.05,1356.25,1360.00,1364.10,1314243.0,17928.09,NaN


In [70]:
NIFTY_CCI

Open     High      Low     Last    Close  \
Stock        Date                                                      
NSE/AXISBANK 2013-01-01  1369.70  1371.50  1360.50  1362.00  1362.70   
             2013-01-02  1371.15  1394.40  1366.65  1386.75  1386.85   
             2013-01-03  1394.00  1396.50  1383.25  1390.00  1389.95   
             2013-01-04  1382.45  1386.95  1370.00  1377.25  1378.95   
             2013-01-07  1385.00  1387.70  1365.50  1366.00  1372.85   
             2013-01-08  1368.80  1369.00  1343.05  1362.70  1360.30   
             2013-01-09  1352.25  1361.95  1344.10  1351.00  1349.65   
             2013-01-10  1359.00  1376.80  1345.50  1369.15  1368.00   
             2013-01-11  1372.10  1374.05  1356.25  1360.00  1364.10   
             2013-01-14  1367.10  1390.90  1361.55  1388.30  1386.10   
             2013-01-15  1394.00  1431.00  1390.00  1428.00  1424.30   
             2013-01-16  1430.00  1430.00  1403.15  1412.20  1409.45   
             2013-01-17  1411.25  1419.00  1394.10  1404.10  1406.40   
             2013-01-18  1410.00  1415.50  1380.20  1384.40  1388.05   
             2013-01-21  1385.15  1391.70  1370.75  1385.00  1385.45   
             2013-01-22  1385.00  1405.80  1380.40  1393.95  1393.40   
             2013-01-23  1402.00  1407.00  1380.45  1407.00  1398.40   
             2013-01-24  1401.20  1409.00  1386.55  1395.00  1396.30   
             2013-01-25  1393.00  1401.80  1379.90  1389.00  1386.25   
             2013-01-28  1381.00  1433.90  1375.85  1416.50  1412.95   
             2013-01-29  1416.30  1503.55  1416.00  1473.35  1475.45   
             2013-01-30  1477.50  1516.05  1476.80  1501.00  1501.25   
             2013-01-31  1494.90  1510.00  1489.95  1506.00  1505.15   
             2013-02-01  1501.10  1515.00  1486.60  1510.00  1509.65   
             2013-02-04  1508.00  1515.00  1475.00  1479.50  1477.70   
             2013-02-05  1470.30  1488.50  1453.10  1456.60  1459.15   
             2013-02-06  1467.85  1470.00  1456.15  1465.55  1460.10   
             2013-02-07  1457.10  1482.00  1456.00  1460.10  1460.55   
             2013-02-08  1459.00  1470.85  1445.55  1457.00  1450.80   
             2013-02-11  1453.25  1499.00  1453.25  1494.90  1494.10   
...                          ...      ...      ...      ...      ...   
NSE/YESBANK  2017-11-20   309.45   314.00   308.20   313.70   313.35   
             2017-11-21   316.00   318.00   310.70   312.60   312.30   
             2017-11-22   315.60   315.80   307.00   310.00   309.75   
             2017-11-23   310.00   316.35   307.15   315.40   313.25   
             2017-11-24   315.00   315.95   312.05   312.50   313.05   
             2017-11-27   312.00   316.00   309.50   315.05   315.30   
             2017-11-28   314.50   316.90   313.65   314.35   315.00   
             2017-11-29   315.00   316.00   310.95   312.50   312.90   
             2017-11-30   311.00   311.95   305.00   306.90   306.90   
             2017-12-01   307.80   309.90   305.10   307.05   306.85   
             2017-12-04   307.80   308.20   301.65   303.95   303.20   
             2017-12-05   302.15   310.00   300.60   309.25   308.70   
             2017-12-06   306.80   309.90   304.60   306.65   306.45   
             2017-12-07   306.55   311.95   306.55   310.55   310.50   
             2017-12-08   312.00   315.50   311.00   312.50   312.70   
             2017-12-11   313.50   315.80   310.60   310.90   311.60   
             2017-12-12   312.00   312.00   305.90   306.35   306.80   
             2017-12-13   306.35   307.35   301.05   302.25   301.90   
             2017-12-14   303.90   304.65   301.75   304.10   303.90   
             2017-12-15   307.00   317.45   307.00   316.05   315.90   
             2017-12-18   315.90   318.00   305.30   311.80   311.15   
             2017-12-19   314.00   314.40   309.55   312.40   312.25   
             2017-12-20   312.40   313.40   310.00   311.20   311.70